In [1]:
# Code aus "Neuronale Netze selbst programmieren,
# ein verständlicher Einstieg mit Python"
# von Tariq Rashid , O'Reilly
# license GPLv2

import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
import matplotlib.pyplot
%matplotlib inline
import os
# helper to load data from PNG image files
import imageio
# glob helps select multiple files using patterns
import glob
# saving the NN 
import pickle
#  !pip install dill
import dill

In [2]:
# neural network class definition
class neuralNetwork:

    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each inpzt, hidden oputput layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # learning rate
        self.lr = learningrate

        # link weight matrices, wih and who 
        # weights inside the arrays aer w_i_j, where link is from node i to node j in the next layer
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # acitivation function is the sigmoid funciton
        #self.activation_function = lambda x: scipy.special.expit(x)
        #self.inverse_activation_function = lambda x: scipy.special.logit(x)
        
        # performance 
        self.performance = 0
        
        # epochs
        self.epochs = 0

        pass
    
    def activation_function(self, arr):
        arrX = 0
        for x in arr:
            arrY = 0
            for y in x:
                arr[arrX][arrY] = scipy.special.expit(y)
                arrY += 1
                pass
            arrX += 1
            pass
        return arr
    
    def inverse_activation_function(self, arr):
        arrX = 0
        for x in arr:
            arrY = 0
            for y in x:
                arr[arrX][arrY] = scipy.special.logit(y)
                arrY += 1
                pass
            arrX += 1
            pass
        return arr
    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer        
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)      
        
        # calculate signals final output layer        
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

    
    # backquery the neural network
    # we'll use the same termnimology to each item, 
    # eg target are the values at the right of the network, albeit used as input
    # eg hidden_output is the signal to the right of the middle nodes
    def backquery(self, targets_list):
        # transpose the targets list to a vertical array
        final_outputs = numpy.array(targets_list, ndmin=2).T
        
        # calculate the signal into the final output layer
        final_inputs = self.inverse_activation_function(final_outputs)

        # calculate the signal out of the hidden layer
        hidden_outputs = numpy.dot(self.who.T, final_inputs)
        # scale them back to 0.01 to .99
        hidden_outputs -= numpy.min(hidden_outputs)
        hidden_outputs /= numpy.max(hidden_outputs)
        hidden_outputs *= 0.98
        hidden_outputs += 0.01
        
        # calculate the signal into the hidden layer
        hidden_inputs = self.inverse_activation_function(hidden_outputs)
        
        # calculate the signal out of the input layer
        inputs = numpy.dot(self.wih.T, hidden_inputs)
        # scale them back to 0.01 to .99
        inputs -= numpy.min(inputs)
        inputs /= numpy.max(inputs)
        inputs *= 0.98
        inputs += 0.01
        
        return inputs

In [3]:
# https://stackoverflow.com/questions/7165749/open-file-in-a-relative-location-in-python
# relative path to files
fileDir = os.path.dirname(os.path.realpath('__file__'))
# For accessing the file in the parent folder of the current folder
# small test data for quick calculations
#training_data_path = os.path.join(fileDir, '../../trainingdata/testdata_mnist/mnist_train_100.csv')
#test_data_path = os.path.join(fileDir, '../../trainingdata/testdata_mnist/mnist_test_10.csv')
# real test data (60'000 training data / 10'000 test data)
training_data_path = os.path.join(fileDir, '../../trainingdata/testdata_mnist/mnist_train.csv')
test_data_path = os.path.join(fileDir, '../../trainingdata/testdata_mnist/mnist_test.csv') 
trained_NN_Path = os.path.join(fileDir, 'Trained_NN/Numbers/') 

In [4]:
# load mnist training csv file into a list
training_data_file = open(training_data_path, 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [5]:
# load mnist test csv file into a list
test_data_file = open(test_data_path, 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [6]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes_list = [10,20,30,40,50,60,70,80,90,100,120,140,160,180,200,250,300,350,400,500,800,1000,2000]
output_nodes = 10

# learning rate
learning_rate_list = [0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

# epochs - how often to train with the training data
epochs_list = [1,2,3,4,5,6,7]




In [7]:
# neural network teststand
class nnTestStand:

    
    # initialise the teststand
    def __init__(self, neuralNetwork, epochs):
        self.nn = neuralNetwork
        self.epochs = epochs
    
    # train the neural network
    def trainNN(self):
        self.nn.epochs = self.epochs

        # epochs is the number of times the training data set is used for training
        for e in range(self.epochs):

            # go through all records in the training data set
            for record in training_data_list:
                # split the record by the ',' commas
                all_values = record.split(',')
                # scale input to range 0.01 to 1.00
                inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
                # create the target output values (all 0.01, except the desired label which is 0.99)
                targets = numpy.zeros(self.nn.onodes) + 0.01
                # all_values[0] is the target label for this record
                targets[int(all_values[0])] = 0.99
                self.nn.train(inputs, targets)
                pass
            pass
        pass
    
    
    # test the neural network
    def testNN(self):
        # scorecard for how well the network performs
        scorecard = []

        # go through all the records in the test data set
        for record in test_data_list:
            # split the record by the ',' commas
            all_values = record.split(',')
            # correct answer is first value
            correct_label = int(all_values[0])
            # scale input to range 0.01 to 1.00
            inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
            # query the network
            outputs = self.nn.query(inputs)
            # the index of the highest value corresponds to the label
            label = numpy.argmax(outputs)
            # append correct or incorrect to list
            if (label == correct_label):
                # network's answer matches correct answer, add 1 to scorecard
                scorecard.append(1)
            else:
                # network's answer doesn't match correct answer, add 0 to scorecard
                scorecard.append(0)
                pass

            pass

        # calculate the performance score (correct answers/ all answers)
        scorecard_array = numpy.asarray(scorecard)
        performance = scorecard_array.sum() / scorecard_array.size

        # save in the NN object
        self.nn.performance = performance

        pass
    
    # open object in pickle from: https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence
    def saveBetterPerformingNN(self):
        # create filename and path 
        filename = "numberNN_%snodes_%slr_%sepochs.nn"%(self.nn.hnodes, self.nn.lr, self.nn.epochs)
        fullFilePath = trained_NN_Path + filename
        print(filename)

        # check if the file already exists - the nn was calculated the second time
        if(os.path.isfile(fullFilePath)):
            # if yes - open the object
            with open(fullFilePath, 'rb') as input:
                nnOld = pickle.load(input)

            # check if we have a higer precision 
            # if yes - save the new
            # if no - nothing happens we keep the old one
            print("old performance: %s   new Performance: %s"%(nnOld.performance, self.nn.performance))
            if(self.nn.performance > nnOld.performance):
                # if yes - save the new
                self.saveNN(fullFilePath)
                pass
            else:
                pass
            
        else:
            # save the new calculated nn
            self.saveNN(fullFilePath)
            print("new Performance: %s"%(self.nn.performance))
            pass

        pass
    
    
    # save object in pickle from: https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence
    def saveNN(self, path):
        # create the file
        with open(path, 'wb') as output:
            # save the object
            pickle.dump(self.nn, output, pickle.HIGHEST_PROTOCOL)
        pass

In [ ]:
# repeat more than one time to find better NNs
reputations = 1
for reputation in range(reputations):

    # go through all combinations for training (hidden_nodes x learning_rate x epochs)
    for hidden_nodes in hidden_nodes_list:
        for learning_rate in learning_rate_list:
            for epochs in epochs_list:
                # create instance of neural network
                n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
                # create a teststand for the neural network
                teststand = nnTestStand(n, epochs)
                # train the network
                teststand.trainNN()
                # test the NN / its performance
                teststand.testNN()
                # save the NN if it perfomered better then before
                teststand.saveBetterPerformingNN()      
                pass
            pass
        pass
    pass


numberNN_10nodes_0.01lr_1epochs.nn
new Performance: 0.7749
numberNN_10nodes_0.01lr_2epochs.nn
new Performance: 0.8631
numberNN_10nodes_0.01lr_3epochs.nn
new Performance: 0.8807
numberNN_10nodes_0.01lr_4epochs.nn
new Performance: 0.8844
numberNN_10nodes_0.01lr_5epochs.nn
new Performance: 0.8926
numberNN_10nodes_0.01lr_6epochs.nn
new Performance: 0.8858
numberNN_10nodes_0.01lr_7epochs.nn
new Performance: 0.899
numberNN_10nodes_0.1lr_1epochs.nn
new Performance: 0.8603
numberNN_10nodes_0.1lr_2epochs.nn
new Performance: 0.8913
numberNN_10nodes_0.1lr_3epochs.nn
new Performance: 0.8917
numberNN_10nodes_0.1lr_4epochs.nn
new Performance: 0.887
numberNN_10nodes_0.1lr_5epochs.nn
new Performance: 0.8868
numberNN_10nodes_0.1lr_6epochs.nn
new Performance: 0.8962
numberNN_10nodes_0.1lr_7epochs.nn
new Performance: 0.8983
numberNN_10nodes_0.2lr_1epochs.nn
new Performance: 0.8677
numberNN_10nodes_0.2lr_2epochs.nn
new Performance: 0.8942
numberNN_10nodes_0.2lr_3epochs.nn
new Performance: 0.8836
numberNN_

numberNN_30nodes_0.01lr_3epochs.nn
new Performance: 0.9277
numberNN_30nodes_0.01lr_4epochs.nn
new Performance: 0.931
numberNN_30nodes_0.01lr_5epochs.nn
new Performance: 0.9379
numberNN_30nodes_0.01lr_6epochs.nn
new Performance: 0.9367
numberNN_30nodes_0.01lr_7epochs.nn
new Performance: 0.937
numberNN_30nodes_0.1lr_1epochs.nn
new Performance: 0.918
numberNN_30nodes_0.1lr_2epochs.nn
new Performance: 0.9332
numberNN_30nodes_0.1lr_3epochs.nn
new Performance: 0.9302
numberNN_30nodes_0.1lr_4epochs.nn
new Performance: 0.9404
numberNN_30nodes_0.1lr_5epochs.nn
new Performance: 0.9408
numberNN_30nodes_0.1lr_6epochs.nn
new Performance: 0.9395
numberNN_30nodes_0.1lr_7epochs.nn
new Performance: 0.9397
numberNN_30nodes_0.2lr_1epochs.nn
new Performance: 0.9168
numberNN_30nodes_0.2lr_2epochs.nn
new Performance: 0.919
numberNN_30nodes_0.2lr_3epochs.nn
new Performance: 0.9324
numberNN_30nodes_0.2lr_4epochs.nn
new Performance: 0.9267
numberNN_30nodes_0.2lr_5epochs.nn
new Performance: 0.9356
numberNN_30no

numberNN_50nodes_0.01lr_5epochs.nn
new Performance: 0.9409
numberNN_50nodes_0.01lr_6epochs.nn
new Performance: 0.947
numberNN_50nodes_0.01lr_7epochs.nn
new Performance: 0.9499
numberNN_50nodes_0.1lr_1epochs.nn
new Performance: 0.9369
numberNN_50nodes_0.1lr_2epochs.nn
new Performance: 0.9475
numberNN_50nodes_0.1lr_3epochs.nn
new Performance: 0.9539
numberNN_50nodes_0.1lr_4epochs.nn
new Performance: 0.9534
numberNN_50nodes_0.1lr_5epochs.nn
new Performance: 0.9568
numberNN_50nodes_0.1lr_6epochs.nn
new Performance: 0.9578
numberNN_50nodes_0.1lr_7epochs.nn
new Performance: 0.9582
numberNN_50nodes_0.2lr_1epochs.nn
new Performance: 0.9368
numberNN_50nodes_0.2lr_2epochs.nn
new Performance: 0.9379
numberNN_50nodes_0.2lr_3epochs.nn
new Performance: 0.9385
numberNN_50nodes_0.2lr_4epochs.nn
new Performance: 0.9469
numberNN_50nodes_0.2lr_5epochs.nn
new Performance: 0.9455
numberNN_50nodes_0.2lr_6epochs.nn
new Performance: 0.949
numberNN_50nodes_0.2lr_7epochs.nn
new Performance: 0.9443
numberNN_50no

numberNN_70nodes_0.01lr_7epochs.nn
new Performance: 0.9569
numberNN_70nodes_0.1lr_1epochs.nn
new Performance: 0.9448
numberNN_70nodes_0.1lr_2epochs.nn
new Performance: 0.9562
numberNN_70nodes_0.1lr_3epochs.nn
new Performance: 0.9568
numberNN_70nodes_0.1lr_4epochs.nn
new Performance: 0.9626
numberNN_70nodes_0.1lr_5epochs.nn
new Performance: 0.9613
numberNN_70nodes_0.1lr_6epochs.nn
new Performance: 0.9653
numberNN_70nodes_0.1lr_7epochs.nn
new Performance: 0.9619
numberNN_70nodes_0.2lr_1epochs.nn
new Performance: 0.9376
numberNN_70nodes_0.2lr_2epochs.nn
new Performance: 0.9561
numberNN_70nodes_0.2lr_3epochs.nn
new Performance: 0.9513
numberNN_70nodes_0.2lr_4epochs.nn
new Performance: 0.9592
numberNN_70nodes_0.2lr_5epochs.nn
new Performance: 0.957
numberNN_70nodes_0.2lr_6epochs.nn
new Performance: 0.9557
numberNN_70nodes_0.2lr_7epochs.nn
new Performance: 0.9549
numberNN_70nodes_0.3lr_1epochs.nn
new Performance: 0.9403
numberNN_70nodes_0.3lr_2epochs.nn
new Performance: 0.9412
numberNN_70nod

numberNN_90nodes_0.1lr_2epochs.nn
new Performance: 0.9612
numberNN_90nodes_0.1lr_3epochs.nn
new Performance: 0.9664
numberNN_90nodes_0.1lr_4epochs.nn
new Performance: 0.9619
numberNN_90nodes_0.1lr_5epochs.nn
new Performance: 0.9661
numberNN_90nodes_0.1lr_6epochs.nn
new Performance: 0.9649
numberNN_90nodes_0.1lr_7epochs.nn
new Performance: 0.9647
numberNN_90nodes_0.2lr_1epochs.nn
new Performance: 0.9453
numberNN_90nodes_0.2lr_2epochs.nn
new Performance: 0.9565
numberNN_90nodes_0.2lr_3epochs.nn
new Performance: 0.9609
numberNN_90nodes_0.2lr_4epochs.nn
new Performance: 0.9602
numberNN_90nodes_0.2lr_5epochs.nn
new Performance: 0.9636
numberNN_90nodes_0.2lr_6epochs.nn
new Performance: 0.9586
numberNN_90nodes_0.2lr_7epochs.nn
new Performance: 0.953
numberNN_90nodes_0.3lr_1epochs.nn
new Performance: 0.9384
numberNN_90nodes_0.3lr_2epochs.nn
new Performance: 0.9523
numberNN_90nodes_0.3lr_3epochs.nn
new Performance: 0.9537
numberNN_90nodes_0.3lr_4epochs.nn
new Performance: 0.9503
numberNN_90node